<a href="https://colab.research.google.com/github/BrutexBr/case-livup-ultimo/blob/main/caselivup_ultimo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from IPython import get_ipython
from IPython.display import display
# %%
# Instalando pacotes necessários
!pip install openpyxl --quiet

# Importando bibliotecas
import pandas as pd

In [6]:
# Parâmetros iniciais
clientes_iniciais = 100
CAC = 150
ticket_primeira_compra = 100
ticket_recompra = 200
margem_lucro = 0.3  # Margem de contribuição (receita - custo variável)
prob_recompra_inicial = 0.2
prob_recompra_final = 0.05  # Taxa fixa a partir do mês 10
custo_operacional_inicial = 7000
custo_mes_0 = 22000  # Inclui CAC + custos operacionais
meses_analise = 30  # Total de meses de análise (alterado para 30)

In [7]:
# Probabilidade de recompra em porcentagem (meses 2 a 9)
prob_recompra_variavel = [
    prob_recompra_inicial - (i * (prob_recompra_inicial - prob_recompra_final) / 8) for i in range(8)
]

# Variáveis de controle
lucro_ou_prejuizo_acumulado = 0  # Lucro acumulado inicial: 0

# Lista para armazenar os resultados
lucros_mensais = []

# Mês 0: Compra inicial
receita_primeira_compra = clientes_iniciais * ticket_primeira_compra
margem_mes_zero = receita_primeira_compra * margem_lucro  # Margem de contribuição
lucro_ou_prejuizo_acumulado = -custo_mes_0 + receita_primeira_compra  # Lucro acumulado do mês 0 = Custo do Mês - Receita do Mês


lucros_mensais.append(
    (
        0,
        clientes_iniciais,
        receita_primeira_compra,
        -custo_mes_0,  # Custo do mês 0 (incluindo CAC)
        margem_mes_zero,
        lucro_ou_prejuizo_acumulado,
        0,  # Probabilidade de recompra para o mês 0
    )
)


In [8]:
# Cálculo para os meses 1 a 30
for mes in range(1, meses_analise + 1):
    if mes == 1:
        prob_recompra_mes = prob_recompra_inicial
    elif 2 <= mes <= 9:
        prob_recompra_mes = prob_recompra_variavel[mes - 2]
    else:
        prob_recompra_mes = prob_recompra_final

    # Clientes recomprando (relativo aos 100 clientes iniciais)
    clientes_recomprando = int(clientes_iniciais * prob_recompra_mes)

    # Receita, custos e margem
    receita_mes = clientes_recomprando * ticket_recompra
    custo_mes = -(receita_mes * (1 - margem_lucro))  # Custo do mês (negativo)
    margem = receita_mes - (-custo_mes)  # Margem = Receita - Custo (custo é negativo)
    lucro_ou_prejuizo_acumulado = lucro_ou_prejuizo_acumulado + custo_mes + receita_mes  # Lucro/prejuízo = Lucro/Prejuízo Acumulado Anterior + Custo do Mês - Receita do Mês


    # Armazenar resultados
    lucros_mensais.append(
        (
            mes,
            clientes_recomprando,
            receita_mes,
            custo_mes,  # Custo do mês (negativo)
            margem,
            lucro_ou_prejuizo_acumulado,
            prob_recompra_mes * 100,  # Probabilidade de recompra em %
        )
    )

In [9]:
# Criando o DataFrame com os resultados
df_resultados = pd.DataFrame(
    lucros_mensais,
    columns=[
        "Mês",
        "Clientes Ativos",
        "Receita do Mês",
        "Custo do Mês",
        "Margem de Contribuição",  # Renomeando a coluna "Margem"
        "Lucro ou Prejuízo Acumulado",
        "Probabilidade de Recompra (%)",
    ],
)

# Formatando o DataFrame
def formatar_df(df):
    # Formatação condicional para realçar valores negativos em vermelho
    return df.style.applymap(lambda x: 'color: red' if x < 0 else '', subset=['Custo do Mês', 'Lucro ou Prejuízo Acumulado']) \
                   .format({'Receita do Mês': '{:,.2f}', 'Custo do Mês': '{:,.2f}', 'Margem de Contribuição': '{:,.2f}', 'Lucro ou Prejuízo Acumulado': '{:,.2f}', 'Probabilidade de Recompra (%)': '{:.2f}%'}) \
                   .set_properties(**{'text-align': 'right'})

# Aplicando a formatação
formatted_df = formatar_df(df_resultados)

# Exibindo o DataFrame formatado no Colab
display(formatted_df)

# Exportando para Excel
file_path = "Resultados_Probabilidade_Relativa.xlsx"
df_resultados.to_excel(file_path, index=False, engine="openpyxl")

# Exibindo o local do arquivo gerado
print(f"Arquivo Excel gerado: {file_path}")


<ipython-input-9-68a9e5e4785f>:18: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  return df.style.applymap(lambda x: 'color: red' if x < 0 else '', subset=['Custo do Mês', 'Lucro ou Prejuízo Acumulado']) \


,Mês,Clientes Ativos,Receita do Mês,Custo do Mês,Margem de Contribuição,Lucro ou Prejuízo Acumulado,Probabilidade de Recompra (%)
0,0,100,"10,000.00","-22,000.00","3,000.00","-12,000.00",0.00%
1,1,20,"4,000.00","-2,800.00","1,200.00","-10,800.00",20.00%
2,2,20,"4,000.00","-2,800.00","1,200.00","-9,600.00",20.00%
3,3,18,"3,600.00","-2,520.00","1,080.00","-8,520.00",18.13%
4,4,16,"3,200.00","-2,240.00",960.00,"-7,560.00",16.25%
5,5,14,"2,800.00","-1,960.00",840.00,"-6,720.00",14.37%
6,6,12,"2,400.00","-1,680.00",720.00,"-6,000.00",12.50%
7,7,10,"2,000.00","-1,400.00",600.00,"-5,400.00",10.62%
8,8,8,"1,600.00","-1,120.00",480.00,"-4,920.00",8.75%
9,9,6,"1,200.00",-840.00,360.00,"-4,560.00",6.87%


Arquivo Excel gerado: Resultados_Probabilidade_Relativa.xlsx


In [10]:
# Faz o download do arquivo Excel
from google.colab import files
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>